In [1]:
import h5py as h5
import arepo
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm import tqdm
from scipy.interpolate import interp1d
from scipy.stats import binned_statistic_2d
from numba import njit
from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u

from joblib import Parallel,delayed

basepath = '/n/holylfs05/LABS/hernquist_lab/Users/abeane/Mgdec/'

import sys
sys.path.append(basepath+'note/')

import illustris_python as il
TNGbase = '/n/holylfs05/LABS/hernquist_lab/IllustrisTNG/Runs/L35n2160TNG/output/'

from lowda import load_galaxy
import lowda as lwd
from scipy.spatial import KDTree

In [2]:
def get_time(time, redshift=False, 
             Omega0=0.3089, 
             OmegaLambda=0.6911,
             HubbleParam=0.6774):
    HUBBLE = 3.2407789e-18
    SEC_PER_MEGAYEAR = 3.15576e13
    
    if redshift:
        a = 1./(1.+time)
    else:
        a = time
    
    fac = 2. / (3. * np.sqrt(OmegaLambda))
    ans = fac * np.arcsinh(np.sqrt(a**3 * OmegaLambda/Omega0))

    ans /= HUBBLE * HubbleParam
    ans /= SEC_PER_MEGAYEAR * 1000
    
    return ans

In [3]:
# Define a class to hold the header information
class SnapshotHeader:
    def __init__(self, header):
        self.BoxSize = header.BoxSize
        self.Composition_vector_length = header.Composition_vector_length
        self.Flag_Cooling = header.Flag_Cooling
        self.Flag_DoublePrecision = header.Flag_DoublePrecision
        self.Flag_Feedback = header.Flag_Feedback
        self.Flag_Metals = header.Flag_Metals
        self.Flag_Sfr = header.Flag_Sfr
        self.Flag_StellarAge = header.Flag_StellarAge
        self.Git_commit = header.Git_commit
        self.Git_date = header.Git_date
        self.HubbleParam = header.HubbleParam
        self.MassTable = header.MassTable
        self.NumFilesPerSnapshot = header.NumFilesPerSnapshot
        self.NumPart_ThisFile = header.NumPart_ThisFile
        self.NumPart_Total = header.NumPart_Total
        self.NumPart_Total_HighWord = header.NumPart_Total_HighWord
        self.Omega0 = header.Omega0
        self.OmegaBaryon = header.OmegaBaryon
        self.OmegaLambda = header.OmegaLambda
        self.Redshift = header.Redshift
        self.Time = header.Time
        self.UnitLength_in_cm = header.UnitLength_in_cm
        self.UnitMass_in_g = header.UnitMass_in_g
        self.UnitVelocity_in_cm_per_s = header.UnitVelocity_in_cm_per_s

# Initialize the meta dictionary
meta = {}
meta['snap_list'] = np.arange(100)
meta['header'] = []
meta['redshift'] = []
meta['scale_factor'] = []
meta['time'] = []
meta['time_lookback'] = []

# Reference time
t0 = get_time(1.)

# Populate the meta dictionary with headers and other info for each snapshot
for i in meta['snap_list']:
    header = arepo.Snapshot(TNGbase, i, onlyHeader=True)
    
    # Create a new SnapshotHeader instance with the header data
    header_info = SnapshotHeader(header)
    
    # Store header information in meta['header']
    meta['header'].append(header_info)
    
    # Store other attributes in the respective arrays
    meta['redshift'].append(header_info.Redshift)
    meta['scale_factor'].append(header_info.Time)
    meta['time'].append(get_time(header_info.Time))
    meta['time_lookback'].append(t0 - get_time(header_info.Time))

# Store HubbleParam as an example of accessing a header property
meta['HubbleParam'] = meta['header'][0].HubbleParam

# Convert lists to numpy arrays for numerical operations
meta['redshift'] = np.array(meta['redshift'])
meta['scale_factor'] = np.array(meta['scale_factor'])
meta['time'] = np.array(meta['time'])
meta['time_lookback'] = np.array(meta['time_lookback'])

In [4]:
@njit
def rodrigues_formula(k, v, theta):
    N = v.shape[0]
    v_rot = np.zeros(np.shape(v))
    
    ctheta = np.cos(theta)
    stheta = np.sin(theta)
    
    for i in range(N):
        v_rot[i] = v[i] * ctheta + np.cross(k, v[i]) * stheta + k * (np.dot(k, v[i])) * (1-ctheta)
    
    return v_rot

In [5]:
def get_rot_pos_vel(subID, snapnum, ptype=4, rhalf_fac=2,
                    k=None, theta=None, return_k_theta=False, key_rhalf=10, Nmin=4000):
    # get COM, COMV, and ang mom of stars
    # print('loading snapnum=', snapnum, 'subID=', subID)
    subhalo = il.groupcat.loadSingle(TNGbase, snapnum, subhaloID=subID)
    # print('done loading snapnum=', snapnum, 'subID=', subID)
    # print('halfmass rad type=', subhalo['SubhaloHalfmassRadType'])
    grpID = subhalo['SubhaloGrNr']
    snap = {}
    snap[4] = il.snapshot.loadHalo(TNGbase, snapnum, grpID, 4)
    if ptype != 4:
        snap[ptype] = il.snapshot.loadHalo(TNGbase, snapnum, grpID, ptype)
    
    COM = subhalo['SubhaloPos']
    # print(np.median(snap[4]['Coordinates'], axis=0), COM)
    
    pos = snap[4]['Coordinates'] - COM
    r = np.linalg.norm(pos, axis=1)
    
    rhalf = subhalo['SubhaloHalfmassRadType'][4]

    in_rhalf = r < rhalf_fac * rhalf
    is_star = snap[4]['GFM_StellarFormationTime'] > 0
    is_star_in_rhalf = np.logical_and(is_star, in_rhalf)
    
    # for later, all stars in key_rhalf
    rpt = np.linalg.norm(snap[ptype]['Coordinates'] - COM, axis=1)
    in_key_rhalf = rpt < key_rhalf * rhalf
    if ptype==4:
        in_key_rhalf = np.logical_and(is_star, in_key_rhalf)
    
    if np.sum(is_star_in_rhalf) < Nmin:
        return None, None, None, None
    
    vel_in_rhalf = snap[4]['Velocities'][is_star_in_rhalf]
    mass_in_rhalf = snap[4]['Masses'][is_star_in_rhalf]
    # print(rhalf, np.sum(is_star_in_rhalf), np.sum(mass_in_rhalf))
    COMV = np.average(vel_in_rhalf, axis=0, weights=mass_in_rhalf)
    
    vel = snap[4]['Velocities'] - COMV
    
    ang = np.cross(pos[is_star_in_rhalf], vel[is_star_in_rhalf])
    ang *= mass_in_rhalf.reshape(-1, 1)
    ang = np.sum(ang, axis=0)
    
    ang_mom = ang

    angmom_dir = ang_mom/np.linalg.norm(ang_mom)
    if theta is None:
        theta = np.arccos(np.dot(angmom_dir, np.array([0, 0, 1])))
    if k is None:
        k = np.cross(ang_mom, np.array([0, 0, 1.]))
        k /= np.linalg.norm(k)
    
    pos = snap[ptype]['Coordinates'] - COM
    vel = snap[ptype]['Velocities'] - COMV
    mass = snap[ptype]['Masses']
    
    pos_rot = rodrigues_formula(k, pos.astype(np.float64), theta)
    vel_rot = rodrigues_formula(k, vel.astype(np.float64), theta)
    
    pos_rot *= meta['header'][snapnum].Time/meta['header'][snapnum].HubbleParam
    vel_rot *= np.sqrt(meta['header'][snapnum].Time)
    
    if return_k_theta:
        return k, theta
    
    return pos_rot, vel_rot, mass, in_key_rhalf

In [41]:
def mdot_eddington(bh_mass):
    GRAVITY = 6.6738e-8
    CLIGHT = 2.99792458e10
    PROTONMASS = 1.67262178e-24
    THOMPSON = 6.65245873e-25
    UnitTime_in_s = meta['header'][99].UnitLength_in_cm / meta['header'][99].UnitVelocity_in_cm_per_s
    
    BlackHoleRadiativeEfficiency = 0.2
    
    ans = 4*np.pi * GRAVITY * CLIGHT * PROTONMASS / (BlackHoleRadiativeEfficiency * CLIGHT * CLIGHT * THOMPSON)
    ans *= UnitTime_in_s / meta['header'][99].HubbleParam
    ans *= np.copy(bh_mass)
    
    return ans

In [20]:
subID = 392276
snapnum = 99

In [21]:
subhalo = il.groupcat.loadSingle(TNGbase, snapnum, subhaloID=subID)
group = il.groupcat.loadSingle(TNGbase, snapnum, haloID=subhalo['SubhaloGrNr'])

In [22]:
subhalo['SubhaloGrNr']

46

In [44]:
snap = {}
fields = {}
fields[0] = ['Coordinates', 'Masses', 'StarFormationRate']
fields[4] = ['Coordinates', 'Masses']
fields[5] = ['Coordinates', 'Masses', 'BH_Mdot', 'BH_MdotEddington']
for pt in tqdm([0, 4, 5]):
    snap[pt] = il.snapshot.loadHalo(TNGbase, snapnum, subhalo['SubhaloGrNr'], pt, fields=fields[pt])

100%|██████████| 3/3 [00:00<00:00,  8.45it/s]


In [12]:
group['Group_M_Mean200'] / meta['HubbleParam']

496.94722403387493

In [16]:
group['Group_R_Mean200']/meta['HubbleParam']

532.2841377359665

In [17]:
list(subhalo.keys())

['SubhaloBHMass',
 'SubhaloBHMdot',
 'SubhaloBfldDisk',
 'SubhaloBfldHalo',
 'SubhaloCM',
 'SubhaloFlag',
 'SubhaloGasMetalFractions',
 'SubhaloGasMetalFractionsHalfRad',
 'SubhaloGasMetalFractionsMaxRad',
 'SubhaloGasMetalFractionsSfr',
 'SubhaloGasMetalFractionsSfrWeighted',
 'SubhaloGasMetallicity',
 'SubhaloGasMetallicityHalfRad',
 'SubhaloGasMetallicityMaxRad',
 'SubhaloGasMetallicitySfr',
 'SubhaloGasMetallicitySfrWeighted',
 'SubhaloGrNr',
 'SubhaloHalfmassRad',
 'SubhaloHalfmassRadType',
 'SubhaloIDMostbound',
 'SubhaloLen',
 'SubhaloLenType',
 'SubhaloMass',
 'SubhaloMassInHalfRad',
 'SubhaloMassInHalfRadType',
 'SubhaloMassInMaxRad',
 'SubhaloMassInMaxRadType',
 'SubhaloMassInRad',
 'SubhaloMassInRadType',
 'SubhaloMassType',
 'SubhaloParent',
 'SubhaloPos',
 'SubhaloSFR',
 'SubhaloSFRinHalfRad',
 'SubhaloSFRinMaxRad',
 'SubhaloSFRinRad',
 'SubhaloSpin',
 'SubhaloStarMetalFractions',
 'SubhaloStarMetalFractionsHalfRad',
 'SubhaloStarMetalFractionsMaxRad',
 'SubhaloStarMetalli

In [33]:
# stellar and gas mass

COM = subhalo['SubhaloPos']
sthalfmass = subhalo['SubhaloHalfmassRadType'][4]
print(sthalfmass)

rdiff0 = np.linalg.norm(snap[0]['Coordinates'] - COM, axis=1)
rdiff4 = np.linalg.norm(snap[4]['Coordinates'] - COM, axis=1)

key0 = rdiff0 < 4 * sthalfmass
key4 = rdiff4 < 4 * sthalfmass

M0 = np.sum(snap[0]['Masses'][key0])/meta['HubbleParam']
M4 = np.sum(snap[4]['Masses'][key4])/meta['HubbleParam']

print('gasmass: ', M0)
print('starmass:', M4)
print('fg:', M0/(M0+M4))

1.98503
gasmass:  0.2561416339480071
starmass: 7.564849757782153
fg: 0.03275053265227331


In [36]:
subhalo['SubhaloSFR']

0.5620243

In [45]:
print(snap[5]['BH_Mdot'][0]/snap[5]['BH_MdotEddington'])

[4.4763976e-05 8.8934976e-05 2.7939538e-04 7.1673311e-04 1.5576323e-03
 2.1341494e-03 8.2967998e-03]
